## 신규 구매 여부 추출

In [1]:
import pandas as pd
import numpy as np

trn = pd.read_csv('/content/drive/MyDrive/kaggle/santander/data/train_ver2.csv')

In [2]:
# 상품 변수 추출 후 저장
prods = trn.columns[24:].tolist()

In [3]:
# 날짜 → 숫자로 변환(첫 달 = 1, 마지막 달 = 18) 해주는 함수
def date_to_int(str_date):
  Y, M, D = [int(a) for a in str_date.strip().split("-")]
  int_date = (int(Y) - 2015) * 12 + int(M)

  return int_date

In [4]:
# 함수 적용 후 저장
trn['int_date'] = trn['fecha_dato'].map(date_to_int).astype(np.int8)

In [6]:
# 데이터 복사 후, int_date에 1을 더해 lag 생성
trn_lag = trn.copy()
trn_lag['int_date'] += 1
trn_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in trn.columns]

In [10]:
# 원본 데이터와 lag 데이터를 ncodper, int_ date 기준으로 병합
# lag 데이터가 1씩 밀려 있으므로 저번 달 제품 정보가 삽입됨
df_trn = trn.merge(trn_lag, on=['ncodpers','int_date'], how='left')

In [12]:
# 불필요한 변수 제거
del trn, trn_lag

In [15]:
# 결측치 값 = 0으로 대체
for prod in prods:
  prev = prod + '_prev'
  df_trn[prev].fillna(0, inplace = True)

In [16]:
# 원본 데이터에서의 제품 보유 여부 - lag 데이터에서의 제품 보유 여부 비교 → 신규 구매 변수 padd
for prod in prods:
  padd = prod + '_add'
  prev = prod + '_prev'
  df_trn[padd] = ((df_trn[prod] == 1) & (df_trn[prev] == 0)).astype(np.int8)

In [17]:
# 신규 구매 변수 추출 후 저장
add_cols = [prod + '_add' for prod in prods]
labels = df_trn[add_cols].copy()
labels.columns = prods

labels.to_csv('/content/drive/MyDrive/kaggle/santander/data/labels.csv', index = False)